In [ ]:
%load_ext autoreload
%autoreload 2
from os.path import join
from tqdm import tqdm
import pandas as pd
import numpy as np
import clin.llm
import clin.eval.med_status
import clin.parse
import openai
openai.api_key_path = '/home/chansingh/.OPENAI_KEY'
from typing import List
results_dir = '../results-mega/'
# results_dir = '../results_tmp/'
from clin.config import PATH_REPO
import datasets
import imodelsx.viz
import imodelsx.process_results
def viz_blue_rows(df):
    return (
        df
        # highlight rows 2 and 4 including the index
        .style.apply(lambda x: ['background: lightblue' if 'self-verified' in x.name else '' for i in x], axis=1)
        # bold index label for row 2
        .apply(lambda x: ['font-weight: bold' if 'self-verified' in x.name else '' for i in x], axis=1)
        .format(precision=3)
    )   
r = imodelsx.process_results.get_results_df(results_dir, use_cached=False)
r = r[[col for col in r.columns if not col in ['checkpoint_verify', 'role_verify']]]
r = r[r.use_megaprompt == 1]
# r = imodelsx.process_results.average_over_seeds(r, experiment_filename=join(PATH_REPO, 'experiments', 'eval_model.py'))

In [ ]:
r['extracted_strs'].iloc[0]

In [ ]:
r_med = r[r.dataset_name == 'medication_status']
dset = datasets.load_dataset('mitclinicalml/clinical-ie', 'medication_status')
df_val = pd.DataFrame.from_dict(dset['validation'])
df = pd.DataFrame.from_dict(dset['test'])
# df = pd.concat([val, test])
nums = np.arange(len(df)).tolist()
np.random.default_rng(seed=13).shuffle(nums)
# dfe = df.iloc[nums]
r_med = clin.eval.med_status.add_status_eval(r_med, df)

cols = {
    'f1___original': 'Medication name, original',
    'f1___ov_pv_ev': 'Medication name, self-verified',
    'status_f1_macro_cond___original': 'Medication status, original',
    'status_f1_macro_cond___sv': 'Medication status, self-verified',
}
rt_med_status = r_med.groupby(['checkpoint'])[list(cols.keys())].mean().rename(columns=cols).round(3).T
rt_med_status_sem = r_med.groupby(['checkpoint'])[list(cols.keys())].sem(ddof=0).rename(columns=cols).round(3).T
viz_blue_rows(rt_med_status)

In [ ]:
r_ebm = r[r.dataset_name == 'ebm']
cols = {
    'f1___original': 'Clinical trial arm, original',
    'f1___ov_pv_ev': 'Clinical trial arm, self-verified',
}
rt_ebm = r_ebm.groupby(['checkpoint'])[list(cols.keys())].mean().rename(columns=cols).round(3).T
rt_ebm_sem = r_ebm.groupby(['checkpoint'])[list(cols.keys())].sem(ddof=0).rename(columns=cols).round(3).T
viz_blue_rows(rt_ebm)

In [ ]:
for k in rt_ebm.columns:
    rt_ebm[k] = rt_ebm[k].astype(str) + "\err{" + rt_ebm_sem[k].astype(str) + "}"
for k in rt_med_status.columns:
    rt_med_status[k] = (
        rt_med_status[k].astype(str) + "\err{" + rt_med_status_sem[k].astype(str) + "}"
    )
rt = pd.concat((rt_ebm, rt_med_status))
rt.columns = rt.columns.map(lambda x: imodelsx.viz.CHECKPOINTS_RENAME_DICT.get(x, x))
viz_blue_rows(rt)

In [ ]:
print(rt.style.format(precision=3).to_latex(hrules=True))